 #### <p style="background-color: #EDE7F6;color:#6600ff;display: inline-block;padding:.6rem;border-radius:.5rem">Import libraries</p>

In [1]:
import numpy as np 
import pandas as pd 
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
import tensorflow as tf
import keras_nlp
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

import os
os.environ["KERAS_BACKEND"] = "tensorflow" 

import keras
import keras_nlp

import tensorflow as tf

import json
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

import plotly.graph_objs as go
import plotly.express as px

 #### <p style="background-color: #EDE7F6;color:#6600ff;display: inline-block;padding:.6rem;border-radius:.5rem">Settings</p>

In [ ]:
pd.options.display.max_rows = 100
pd.options.display.max_columns = 100
pd.options.display.max_colwidth = 1000
warnings.filterwarnings('ignore')

In [2]:
model_type="distil_bert_base_en_uncased"

 #### <p style="background-color: #EDE7F6;color:#6600ff;display: inline-block;padding:.6rem;border-radius:.5rem">Import data files</p>

In [3]:
train_data_path = "train.csv"
train_data = pd.read_csv(train_data_path)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [ ]:
train_data.head()

## <p style="background-color: #EDE7F6; color: #6600ff;margin:0; display:inline-block;padding:.6rem;border-radius:.25rem;">MODEL ENGINEERING</p>

In [4]:
# Split data into training and test sets (80% train, 20% test)
train_data, test_data = train_test_split(train_data, test_size=0.1, random_state=42)

# Define a function to parse the DataFrame
def parse_dataframe(df):
    # Return features (text review) and labels (sentiment label)
    return df["text"].values, df["target"].values

# Convert training and test data to TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices(parse_dataframe(train_data)).batch(32)
test_dataset = tf.data.Dataset.from_tensor_slices(parse_dataframe(test_data)).batch(32)

In [6]:
train_dataset

<_BatchDataset element_spec=(TensorSpec(shape=(None,), dtype=tf.string, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

In [6]:
train_data.tail(3)

,id,keyword,location,text,target
860,1242,blood,NaN,Omron HEM-712C Automatic Blood Pressure Monito...,0
7603,10862,NaN,NaN,Officials say a quarantine is in place at an A...,1
7270,10409,whirlwind,Stamford & Cork (& Shropshire),I moved to England five years ago today. What ...,1


In [7]:
tokenizer = keras_nlp.models.DistilBertTokenizer.from_preset(model_type)
# backbone = keras_nlp.models.DistilBertBackbone.from_preset(model_type)

AttributeError: module 'keras_nlp' has no attribute 'models'

In [ ]:
packer = keras_nlp.layers.MultiSegmentPacker(
    start_value=tokenizer.cls_token_id,
    end_value=tokenizer.sep_token_id,
    pad_value=tokenizer.pad_token_id,
    sequence_length=500
)

def preprocess(x, y):
    token_ids, _ = packer(tokenizer(x))
    x = {
        "token_ids": token_ids,
        "padding_mask": token_ids != tokenizer.pad_token_id,
    }
    return x, y

train_ds = train_dataset.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
val_ds = test_dataset.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)

In [ ]:
from tensorflow.keras import callbacks

early_stopping = callbacks.EarlyStopping(
    min_delta=0.001, # minimium amount of change to count as an improvement
    patience=3, # how many epochs to wait before stopping
    restore_best_weights=True,
)

In [ ]:
inputs = backbone.input
outputs = backbone(inputs)[:, 0, :]
#outputs = keras.layers.Dense(2028,activation='relu')(outputs)
outputs = keras.layers.Dense(1024,activation='relu')(outputs)
outputs = keras.layers.Dense(512,activation='relu')(outputs)
outputs = keras.layers.Dense(256, activation='relu')(outputs)
outputs = keras.layers.Dense(128,activation='relu')(outputs)

outputs = keras.layers.Dropout(0.1)(outputs)

outputs = keras.layers.Dense(2)(outputs)
    
model = keras.Model(inputs, outputs)
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(1e-5),
     metrics= ["accuracy"],
    jit_compile=True,
    
)

history=model.fit(train_ds, validation_data=val_ds,epochs=2)

In [ ]:
history_df = pd.DataFrame(history.history)
history_df.loc[:, ['loss', 'val_loss']].plot();
print("Minimum validation loss: {}".format(history_df['val_loss'].min()))

In [ ]:
# reading train data
test_data_path = "/kaggle/input/nlp-getting-started/test.csv"

# Load data from CSV file into a DataFrame
test_data = pd.read_csv(test_data_path)

test_data.head()



In [ ]:
# Define a function to parse the DataFrame
def parse_dataframe(df):
    # Return only features (text review)
    return df["text"].values

# Convert test data to TensorFlow dataset
test_dataset = tf.data.Dataset.from_tensor_slices(parse_dataframe(test_data)).batch(28)

# Define the preprocess function
def preprocess(x):
    token_ids, _ = packer(tokenizer(x))
    x = {
        "token_ids": token_ids,
        "padding_mask": token_ids != tokenizer.pad_token_id,
    }
    return x

# Map the preprocess function to the test dataset
test_ds = test_dataset.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)


In [ ]:
#test_ds = tf.data.Dataset.from_tensor_slices((test_data["text"]).values).batch(28)
#test_ids = tf.data.Dataset.from_tensor_slices((test_data["id"]).values).batch(28)



In [ ]:
sample_submission = pd.read_csv("/kaggle/input/nlp-getting-started/sample_submission.csv")
sample_submission.head()

In [ ]:
#predictions = model.predict(test_ds)
sample_submission["target"] = np.argmax( model.predict(test_ds), axis=1)

In [ ]:
sample_submission.head()

In [ ]:
sample_submission.describe()

In [ ]:
sample_submission.to_csv("submission.csv", index=False)